np.nan, None gibi birden çok eksik veri türü vardır. Aralarındaki farkı anlamak için [bu yazıyı](https://kegui.medium.com/what-is-the-difference-between-nan-none-pd-nan-and-np-nan-a8ee0532e2eb) okuyabilirsiniz. Ayrıca notebookdaki komutları ve daha fazlasını incelemek için [pandas dökümanına](https://pandas-docs.github.io/pandas-docs-travis/user_guide/missing_data.html) bakabilirsiniz. Başlangıç olarak notebookdaki komutlar yeterlidir. Daha sonra büyük veri setlerini incelediğimizde eksik verilerle çalışmak için daha işlevsel fonksiyonlar göreceğiz.

# Null değerler ile çalışmak

Pandasta ``None`` ve ``NaN`` ifadeler, verinin eksik veya boş değerleri belirtmek için birbirinin yerine kullanılabilir. Pandas veri yapılarındaki boş yereğerleri saptamak, kaldırmak ve değiştirmek için birkaç yararlı yöntem vardır.

- ``isnull()``: Eksik değerleri gösteren bir boolean filtreleme işlemi yapar
- ``notnull()``:``isnull()`` un tam tersi
- ``dropna()``: Verinin filtrelenmiş versiyonunu yani Null değerlerin atıldığı halini getirir.
- ``fillna()``: Verinin eksik değerlerin doldurulmuş bir kopyasını getirir.


### Null değerleri belirlemek
Pandasın, veri yapılarının boş verilerini algılamak için iki kullanışlı yöntemi vardır: ``isnull()`` ve ``notnull()``.
Her ikisi de veriler üzerinde bir boolean maskeleme (masking) ifadesi döndürür. Örneğin:

In [14]:
import pandas as pd
import numpy as np

In [15]:
data = pd.Series([1, np.nan, "merhaba", None])
data

0          1
1        NaN
2    merhaba
3       None
dtype: object

In [16]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [17]:
data[data.notnull()]

0          1
2    merhaba
dtype: object

"isnull()" ve "notnull()" yöntemleri, "DataFrame" için benzer boolean sonuçları döndürür.

### Null değerlerden kurtulmak

Daha önce kullanılan maskelemeye ek olarak, ``dropna()``(NA değerlerini kaldırır) ve ``fillna()`` (NA değerlerini doldurur) yöntemleri vardır.

In [18]:
data

0          1
1        NaN
2    merhaba
3       None
dtype: object

In [21]:
data.dropna(inplace=True)

In [22]:
data

0          1
2    merhaba
dtype: object

DataFrame için daha fazla seçenek vardır.
Örneğin bir ``DataFrame`` ele alalım:

In [23]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])

df


,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Bir ``DataFrame`` den tekil değerler atamayız. NA değerlerin olduğu satırı veya sütunu atmamız mümkündür. Satırı ya da sütunu atmak için seçim yapmanız gerekir. Herhangi bir seçim yapmazsanız ``dropna()`` varsayılan olarak herhangi(``any``) bir boş değerin olduğu satırları atacaktır.

In [29]:
df.dropna()

,0,1,2
1,2.0,3.0,5


NA değerlerin olduğu sütunları atmak için ``axis=1`` ifadesini veya ``axis="columns"`` ifadesini eklemeniz gerekir.

In [28]:
df.dropna(axis=1)

,2
0,2
1,5
2,6


Herhangi bir işlem yapmadığınız zaman, bu şekilde NA değerleri atmak bazı eksik olmayan verileri de atmaya sebep olur. Yani satırda veya sütunda herhangi bir NA değeri varsa direkt o satırdan kurtulur. Bunun yerine tamamı NA değerlerden oluşan satırlardan kurtulmak isteyebilirsiniz.

Varsayılan ifade  ``how='any'`` şeklindedir. Bu nedenle boş değer içeren herhangi bir satır veya sütun atılacaktır. Ayrıca, yalnızca hepsi boş değer olan satır ve sütunlardan kurtulmak için ``how='all'`` ifadesini eklemeniz gerekir.


In [30]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [31]:
df["non_degerler"] = np.nan
df

,0,1,2,non_degerler
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [33]:
df.dropna(axis="columns", how="all")

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Bütün değerleri boş değer olanlardan değil de 2, 3 vb. tane boş değeri olanlardan kurtulmak için "thresh" parametresi bize kolaylık sağlar. "thresh" parametresini satırda veya sütunda kaç tane NaN olmayan değer olmasını istediğimiz şekilde ayarlarız. Örneğin:

In [34]:
df

,0,1,2,non_degerler
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [35]:
df.iloc[2,3] = 15

In [36]:
df

,0,1,2,non_degerler
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,15.0


In [39]:
df.dropna(axis = "columns", inplace=False, thresh=1)

,0,1,2,non_degerler
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,15.0


In [40]:
df.dropna(axis = "columns", inplace=False, thresh=3)

,2
0,2
1,5
2,6


Burada, yalnızca üç veya temiz değer içeren kolon 2 olduğu için diğerlerini atmıştır.

### Eksik değerleri doldurmak
Bazen eksik değerleri atmak yerine düzgün değerlerle doldurmayı isteyebilirsiniz. Çünkü eğer elimizde çok eksik değer varsa veri setimizden o kadar değeri atmak mantıksız olacaktır. Verileri düzgün istatiksel incelemelerle doldurmak makine öğrenmesi modelimizin performansını büyük ölçüde arttıracaktır.

Bu doldurulan değer sıfır-bir gibi tek bir sayı olabilir veya elimizdeki eğerlerden bir tür atama veya enterpolasyon olabilir.

In [42]:
data = pd.Series([1, np.nan, 2, None, 3], index=list("abcde"))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

NA değerleri aşağıdaki şekilde sıfır ile doldurabiliriz.

In [44]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Eksik veriyi, hemen öncesindeki değerle doldurması için aşağıdaki gibi ``method='ffill'`` ifadesini kullanabiliriz.
f-fill (forward-fill) öncesindeki veriyi ileri ata şeklinde düşünebilirsiniz.

In [45]:
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [46]:
data.fillna(method = "ffill") #forward fill

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Ya da sonrasındaki veriyi geri atamasını söylebiliriz. (back-fill)

In [47]:
data.fillna(method = "bfill") #backward fill

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

DataFrame ler için seçenekler aynıdır ancak ek olarak doldurma işlemi yapacağımız ekseni seçebiliriz.

In [50]:
df

,0,1,2,non_degerler
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,15.0


In [52]:
df.fillna(method = "ffill", axis=1) #forwad fill

,0,1,2,non_degerler
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,15.0


In [55]:
df

,0,1,2,non_degerler
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,15.0


In [53]:
df.fillna(method = "bfill", axis=1)

,0,1,2,non_degerler
0,1.0,2.0,2.0,NaN
1,2.0,3.0,5.0,NaN
2,4.0,4.0,6.0,15.0


##### **ÖNEMLİ NOT**: 
MAKİNE ÖĞRENMESİ MODELİ OLUŞTURMAK İÇİN VERİYİ ANALİZ EDERKEN EKSİK DEĞERLERİ BU YÖNTEMLERLE DOLDURMAYIZ. VERİLERİN BİRBİRİ İÇERİSİNDEKİ KORELASYONUNA BAKAR ÇEŞİTLİ İSTATİKSEL ANALİZLER YAPARIZ. BU DERSTE GÖRDÜKLERİMİZ BİR GİRİŞ AMACI İLE HAZIRLANMIŞ İFADELERDİR. AYRINTILI OLARAK EKSİK VERİLERİ DOLDURMA, VERİ ANALİZ ETME İŞLEMLERİNİ ÖNÜMÜZDEKİ DERSLERDE UYGULAMALI OLARAK GÖRECEĞİZ.